In [2]:
import toml
import os
import pandas as pd
import cv2
import torch

In [2]:
dict_toml = toml.load(open('../config.toml'))

In [3]:
image_path = dict_toml["path"]["dataset"]["image"]

In [4]:
#image_df = pd.read_table("../../list_attr_celeba.txt", header=0,sep=" ", index_col=0)

In [5]:
#image_df["Male"]["000001.jpg"]

In [4]:
f = open('../not_detected.txt', 'r')

datalist = f.read().splitlines()
f.close()

In [7]:
#data = [not i in datalist for i in image_df.index]

In [8]:
#image_df[data]["Male"]

In [9]:
#os.listdir("../../voice/train/F/")

In [10]:
"""
for i in os.listdir("../../voice/train/F/VCC2SF1"):
    a = pd.read_pickle("../../voice/train/F/VCC2SF1/"+i)
    print(a.shape)
"""

'\nfor i in os.listdir("../../voice/train/F/VCC2SF1"):\n    a = pd.read_pickle("../../voice/train/F/VCC2SF1/"+i)\n    print(a.shape)\n'

# dataloader

## transform

### voice

In [7]:
class VoiceTrans(object):
    def __init__(self, maxi, mini):
        self.maxi = maxi
        self.mini = mini

    def norm_voice(self, array):
        array -= self.mini
        array /= self.maxi
    
        return array
    
    def cut(self, voice):
        return voice[:, :voice.shape[1]-voice.shape[1]%4]
    
    def __call__(self, sample):
        
        return self.cut(self.norm_voice(sample))

In [12]:
voice = pd.read_pickle("../../voice/train/VCC2SF1/10001.pkl")

In [13]:
#t = VoiceTrans()
#t(voice).shape

### image

In [8]:
class ImageTrans(object):
    def __init__(self):
        pass

    def norm_image(self, array):
        return array/255
    
    def __call__(self, sample):
        
        return self.norm_image(sample).T

In [15]:
#image = cv2.imread("../../image/000001.jpg")

In [16]:
#a = ImageTrans()
#a(image).shape

# dataset

In [4]:
import pathlib
import itertools
import pandas as pd
import numpy as np
import torch

### voice

In [18]:
class VoiceDataset(torch.utils.data.Dataset):
    def __init__(self, path, train, trans):
        p = pathlib.Path(path)
        
        if train:
            self.path = p / "train"
        else:
            self.path = p / "eval"
        
        self.trans = trans
        
        
        self.dir = [i for i in self.path.iterdir() if i.is_dir()]       
        self.file = list(itertools.chain.from_iterable([[j for j in i.iterdir()] for i in self.dir]))
        
        self.data = [pd.read_pickle(i) for i in self.file]
        self.label = [1 if "M" == str(i.parent)[-2] else -1 for i in self.file]
    
        self.datanum = len(self.data)
    
    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        out_data = self.data[idx]
        out_label = self.label[idx]

        if self.trans:
            k = np.concatenate(self.data,axis=1)
            maxi = k.max()
            mini = k.min()
            self.transform = VoiceTrans(maxi, mini)
            
            out_data = torch.tensor([[self.transform(out_data)]], dtype=torch.float32)

        return out_data, out_label

In [19]:
c = VoiceDataset(path="../../voice", train=1, trans=1)

In [20]:
c.__getitem__(0)[1]

1

### face

In [21]:
import random

class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, path, trans):
        p = pathlib.Path(path)
        if trans:
            self.transform = ImageTrans()
        
        self.dir = p
        self.file = [i for i in self.dir.iterdir() if i.suffix == ".jpg"]
        
        
        if self.transform:
            self.data = [self.transform(cv2.imread(str(i))) for i in self.file]
        else:
            self.data = [cv2.imread(str(i)) for i in self.file]
 
        
        image_df = pd.read_table("../../list_attr_celeba.txt", header=0,sep=" ", index_col=0)
        self.label = [image_df["Male"][i.name] for i  in self.file]
        """
        for i in self.file:
            try:
                image_df["Male"][i.name]
            except:
                print(i.name)
        """     
                
        
        self.datanum = len(self.data)
    
    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        out_data = self.data[idx]
        out_label = self.label[idx]

        
        return out_data, out_label
    
    def sample_label(self, male, num):
        l = random.sample([i for i, x in enumerate(self.label) if x==male], num)
        return l
    
    def sample_data(self, male, num):
        label = self.sample_label(male, num)
        data = self.data
        print(data[0].shape)
        return torch.tensor([data[i] for i in label])
    

In [22]:
#f = ImageDataset("../../image", trans=1)

## pair

In [5]:
from random import sample

class PairDataset(torch.utils.data.Dataset):
    
    voice_path: pathlib.PosixPath
    
    
    
    def __init__(self, voice_path, train, image_path):
        # path定義
        p = pathlib.Path(voice_path)
        if train:
            self.voice_path = p / "train"
        else:
            self.voice_path = p / "eval"
        
        self.image_path = pathlib.Path(image_path)
        
        
        voice_dir = [i for i in self.voice_path.iterdir() if i.is_dir()]       
        voice_file = list(itertools.chain.from_iterable([[j for j in i.iterdir()] for i in voice_dir]))
        
        voice_data = [pd.read_pickle(i) for i in voice_file]
        self.voice_label = [1 if "M" == str(i.parent)[-2] else -1 for i in voice_file]
        
        k = np.concatenate(voice_data,axis=1)
        self.voice_transform = VoiceTrans(k.max(), k.min())
        
        
        self.voice_data = [torch.tensor([[self.voice_transform(i)]], dtype=torch.float32) for i in voice_data]
        
        
        
        
        
        
        image_dir = self.image_path
        image_file = [i for i in image_dir.iterdir() if i.suffix == ".jpg"]
        
        
        image_df = pd.read_table(self.image_path / "list_attr_celeba.txt", header=0,sep=" ", index_col=0)
        self.image_label = [image_df["Male"][i.name] for i  in image_file]
        self.image_label_male = [i for i, x in enumerate(self.image_label) if x == 1]
        self.image_label_female= [i for i, x in enumerate(self.image_label) if x == -1]
        
        self.image_transform = ImageTrans()
        
        self.image_data = [torch.tensor(self.image_transform(cv2.imread(str(i))), dtype=torch.float32) for i in image_file]
 
        
    
           
        
        self.datanum = len(self.voice_data)
    
    
    
    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):

        out_voice_data = self.voice_data[idx]
        out_label = self.voice_label[idx]
        
        k = 4
        if out_label == 1:
            c = sample(self.image_label_male, k)
        else:
            c = sample(self.image_label_female, k)
        out_image_data = torch.stack([self.image_data[i] for i in c])
        

        return out_voice_data, out_image_data, out_label

In [9]:
c = PairDataset(voice_path="../../voice", train=True, image_path="../../image")

In [ ]:
d = PairDataset(voice_path="../../voice", train=False, image_path="../../image")

In [26]:
type(pathlib.Path())

pathlib.PosixPath

## dataloader

In [10]:
def collate_fn(batch):
    # batchはDatasetの返り値 (タプル) のリスト
    voices, images = [], []
    for voice, image, label in batch:
        voices.append(voice)
        images.append(image)
        #labels.append(label)
    
    
    # labelsはTensorリストのまま

    return voices, images

In [11]:
batch_size = 64

In [12]:
trainloader = torch.utils.data.DataLoader(c, batch_size = batch_size, shuffle = False, num_workers = 0, collate_fn = collate_fn)

In [30]:
testloader = torch.utils.data.DataLoader(d, batch_size = batch_size, shuffle = False, num_workers = 0, collate_fn = collate_fn)

In [27]:
for voice, image in trainloader:
    break
print(voice[0].shape)
print(image[0].shape)

torch.Size([1, 1, 36, 176])
torch.Size([4, 3, 32, 32])


In [13]:
from cmvc import *

In [14]:
net = Net()

In [15]:
from torch import optim

In [24]:
net.loss(voice[0], image[0])

tensor(43.7029, grad_fn=<AddBackward0>)

In [ ]:
def train_net(net, train_loader,
             optimizer_cls=optim.Adam,
             n_iter=10, device="cpu"):
    train_losses = []
    
    optimizer = optimizer_cls(net.parameters(), lr=0.001)
    for epoch in range(n_iter):
        running_loss = 0.0
        
        net.train()
        
        for i, (xx, yy) in enumerate(train_loader):
            optimizer.zero_grad()
            losses = torch.zeros(1)
            
            for batch in range(len(xx)):
                loss = net.loss(xx[batch], yy[batch])
                losses += loss
            
            losses.backward()
            optimizer.step()
            print(losses.item(), flush=True)
            
            running_loss += losses.item()
        
        
        train_losses.append(running_loss / i)
        
        print(epoch, train_losses[-1], flush=True)

In [ ]:
net.to("cpu")
train_net(net, trainloader)